# Missionaries and Cannibals

The Missionaries and Cannibals problem is usually stated as follows. Three missionaries and there cannibals are on one side of a river, along with a boat that can hold one or two people. Find a way to get everyone to the other side without ever leaving a group of missionaries in one place outnumbered by the cannibals in that place.



## Goal Formulation

The goal for this problem is to find a stratege for the three missionaries and cannibals to cross the river with a boat which can take at most two people, and make sure at every step no missionary at one side is facing more cannibals than missionaries. We assume all the people starts at the RIGHT side of the river.

## Formulating the Problem

### State

We can describe the states using a tuple contains these information:
* The number of missionaries on the left side of the river.
* The number of cannibals on the left side of the river.
* The number of missionaries on the right side of the river.
* The number of cannibals on the right side of the river.
* Where's the boat.

So it's proper to formulate a tuple with 5 numbers:
```
(0, 0, 3, 3, 1)
```
where the numbers obey the sort as presented above. The last number equals 1 indicating the boat is on the right side of the river and 0 indicating on the left side.

So the starting state and the goal state can be presented as:


In [ ]:
from exercises.e3_9_missionaries_and_cannibals import MissionariesAndCannibals
from notebook import psource


ModuleNotFoundError: No module named 'exercises'

In [ ]:
starting_state = (0, 0, 3, 3, 1)
goal_state = (3, 3, 0, 0, 0)


### Actions

At each state, we have actions to drive one or two people to the other side of the river. Take the side into consideration, we have ten actions:


In [ ]:
possible_actions = ['ML', 'MR', 'CL', 'CR', 'MML', 'MMR', 'CCL', 'CCR', 'MCL', 'MCR']


In these actions, 'ML' means the boat take one missionary on the left side to the right side, 'MMR' means the boat will take two missionaries on the right side to the left side, 'MCR' indicating one missionary and one cannibal will be taken from right side to the left, and others are similarly defined.

However, not all the actions can be excute at one state. The straightforward issue is when the boat is on the right side of the river, actions which take people from left side to the right side are not applicatble. Other issues are listed in the code below.

In [ ]:
psource(MissionariesAndCannibals)


## Seaching for the Result

We do breadth first graph search for the solution:

In [ ]:
from search import breadth_first_graph_search


def solve():
    missionaries_and_cannibals = MissionariesAndCannibals(starting_state)
    return breadth_first_graph_search(missionaries_and_cannibals).solution()


And we can get the action list to reach the goal.

In [ ]:
print(solve())
